## Importer les bibliothèques

On utilise Polars (traitements rapides des DataFrames), NumPy (calculs numériques), les stats de proportions de statsmodels et les fonctions statistiques de SciPy :

In [1]:
import numpy as np
import polars as pl
from scipy.stats import chi2_contingency

## Chargement de DM (Population)

On importe les données démographiques pour associer à chaque patient son bras de traitement

In [2]:
try:
    dm1 = pl.read_csv('dm1.csv')
    dm2 = pl.read_csv('dm2.csv')
    dm = pl.concat([dm1, dm2])
    print(f"DM chargé : {dm.height} patients.")
except:
    dm = pl.read_csv('dm.csv')
    print(f"DM chargé (fichier unique) : {dm.height} patients.")

# Nettoyage clé et bras
dm = dm.with_columns([
    pl.col('USUBJID').cast(pl.Utf8).str.strip_chars().str.to_uppercase(),
    pl.col('ARM').cast(pl.Utf8).str.to_uppercase()
])

# Création de la colonne ARM_CLEAN (BUPNAL vs CLON)
dm = dm.with_columns(
    pl.when(pl.col('ARM').str.contains('BUP'))
    .then(pl.lit('BUPNAL'))
    .otherwise(pl.lit('CLON'))
    .alias('ARM_CLEAN')
)

DM chargé : 411 patients.


## Chargement de  DS (Disposition/Rétention)

In [3]:
try:
    # On force le type String pour éviter les erreurs de fusion de colonnes mixtes
    ds1 = pl.read_csv('ds1.csv', infer_schema_length=0)
    ds2 = pl.read_csv('ds2.csv', infer_schema_length=0)
    
    # Harmonisation des colonnes (si DSOCCUR vs DSDECOD)
    if 'DSOCCUR' in ds1.columns and 'DSDECOD' not in ds1.columns: ds1 = ds1.rename({'DSOCCUR': 'DSDECOD'})
    if 'DSOCCUR' in ds2.columns and 'DSDECOD' not in ds2.columns: ds2 = ds2.rename({'DSOCCUR': 'DSDECOD'})
    
    ds = pl.concat([ds1, ds2], how='diagonal')
    print(f"DS chargé : {ds.height} événements.")
except:
    ds = pl.read_csv('ds.csv', infer_schema_length=0)

ds = ds.with_columns(pl.col('USUBJID').cast(pl.Utf8).str.strip_chars().str.to_uppercase())

DS chargé : 1492 événements.


## Chargement de  LB (Tests Urinaires)

On lis les deux fichiers de tests urinaires, puis on les concatène, car chaque fichier correspond à une cohorte différente 

In [4]:
try:
    lb1 = pl.read_csv('lb1.csv')
    lb2 = pl.read_csv('lb2.csv')
    lb = pl.concat([lb1, lb2])
    print(f"LB chargé : {lb.height} résultats.")
except:
    lb = pl.read_csv('lb.csv')

lb = lb.with_columns(pl.col('USUBJID').cast(pl.Utf8).str.strip_chars().str.to_uppercase())

LB chargé : 17977 résultats.


## Selection population ITT

In [5]:
# On ne garde que les patients uniques avec leur bras de traitement
df_itt = dm.select(['USUBJID', 'ARM_CLEAN']).unique(subset=['USUBJID'])

# Calcul des effectifs par bras (N)
n_bup = df_itt.filter(pl.col('ARM_CLEAN') == 'BUPNAL').height
n_clon = df_itt.filter(pl.col('ARM_CLEAN') == 'CLON').height

print(f"Population ITT : {df_itt.height}")
print(f"-> BUP-NX : {n_bup}")
print(f"-> CLONIDINE : {n_clon}")

Population ITT : 411
-> BUP-NX : 233
-> CLONIDINE : 178


## Critère de jugement principal

In [6]:
# Critere de retention (sousrce DS)
# On cherche ceux qui sont sortis au jour 13 ou plus (ou qui ont complété l'étude)
# Conversion de DSSTDY en numérique (car chargé en string via infer_schema_length=0)
ds_clean = ds.with_columns(pl.col('DSSTDY').cast(pl.Float64, strict=False))

retained_ids = ds_clean.filter(
    pl.col('DSSTDY') >= 13
).select('USUBJID').unique()['USUBJID'].to_list()

print(f"Patients retenus (J13+) : {len(retained_ids)}")


# CRITÈRE ABSTINENCE (Source : LB) 
# On cherche les patients avec un test OPIOÏDE POSITIF à la fin (J13+) -> Ce sont des ÉCHECS
opiate_codes = ['OPIATE', 'MORPH', 'METHADON', 'OXYCOD', 'HEROIN'] 

# Filtre : Visite >= 13, Test Opioïde, Résultat Positif
positive_ids = lb.filter(
    (pl.col('VISITNUM') >= 13) &
    (pl.col('LBTESTCD').str.to_uppercase().is_in(opiate_codes)) &
    (pl.col('LBORRES').str.to_uppercase().str.contains('POS'))
).select('USUBJID').unique()['USUBJID'].to_list()

print(f"Patients positifs à la fin : {len(positive_ids)}")


# COMBINAISON (Flag SUCCESS) 
# On ajoute une colonne SUCCESS : 1 si Retenu ET Non Positif, 0 sinon
df_results = df_itt.with_columns(
    pl.when(
        (pl.col('USUBJID').is_in(retained_ids)) &  # Doit être retenu
        (~pl.col('USUBJID').is_in(positive_ids))   # NE DOIT PAS être positif
    )
    .then(1)
    .otherwise(0)
    .alias('SUCCESS')
)

print("Calcul des succès terminé.")

Patients retenus (J13+) : 208
Patients positifs à la fin : 154
Calcul des succès terminé.


## CALCULS STATISTIQUES et génération du tableau 2

In [7]:
# Nombre de succès par bras
succ_bup = df_results.filter((pl.col('ARM_CLEAN') == 'BUPNAL') & (pl.col('SUCCESS') == 1)).height
succ_clon = df_results.filter((pl.col('ARM_CLEAN') == 'CLON') & (pl.col('SUCCESS') == 1)).height

# Proportions
p_bup = succ_bup / n_bup
p_clon = succ_clon / n_clon
diff = p_bup - p_clon

# Intervalle de confiance standard
def get_ci(p, n):
    z = 1.96
    se = np.sqrt((p * (1 - p)) / n)
    return max(0, p - z * se), min(1, p + z * se)

ci_bup = get_ci(p_bup, n_bup)
ci_clon = get_ci(p_clon, n_clon)

# IC de la Différence (Wald)
se_diff = np.sqrt((p_bup * (1 - p_bup) / n_bup) + (p_clon * (1 - p_clon) / n_clon))
ci_diff = (diff - 1.96 * se_diff, diff + 1.96 * se_diff)
ci_diff_str = f"[{ci_diff[0]*100:.1f}% - {ci_diff[1]*100:.1f}%]"



# P-value (Chi-2 via scipy)
# On doit repasser par numpy pour scipy
contingency = [[succ_bup, n_bup - succ_bup], [succ_clon, n_clon - succ_clon]]
chi2, p_val, _, _ = chi2_contingency(contingency)


# --- AFFICHAGE FINAL (Format Rapport) ---
print(f"| Bras      | N   | Rét+Abs   | % Critère principal   | IC 95%                  |")
print(f"|-----------|-----|-----------|-----------------------|-------------------------|")
print(f"| BUPNAL    | {n_bup} | {succ_bup}       | {p_bup*100:.1f}%                 | [{ci_bup[0]*100:.1f}% - {ci_bup[1]*100:.1f}%] |")
print(f"| CLON      | {n_clon} | {succ_clon}        | {p_clon*100:.1f}%                 | [{ci_clon[0]*100:.1f}% - {ci_clon[1]*100:.1f}%] |")
print(f"| **Diff** |     |           | {diff*100:.1f}%                 | [{ci_diff[0]*100:.1f}% - {ci_diff[1]*100:.1f}%]   |")
print(f"\n**p-value (chi2)** : {p_val:.2e}")

# Optionnel : Sauvegarde CSV
df_final = pl.DataFrame({
    "Bras": ["BUPNAL", "CLON", "Diff"],
    "N": [str(n_bup), str(n_clon), ""],
    "Rét+Abs": [str(succ_bup), str(succ_clon), ""],
    "% Critère": [f"{p_bup*100:.1f}%", f"{p_clon*100:.1f}%", f"{diff*100:.1f}%"],
    "IC 95%": [
        f"[{ci_bup[0]*100:.1f}% - {ci_bup[1]*100:.1f}%]",
        f"[{ci_clon[0]*100:.1f}% - {ci_clon[1]*100:.1f}%]",
        f"[{ci_diff[0]*100:.1f}% - {ci_diff[1]*100:.1f}%]"
    ],
    "P-value": [f"{p_val:.2e}", "", ""]
})
df_final.write_csv("Tableau2_Final.csv")

| Bras      | N   | Rét+Abs   | % Critère principal   | IC 95%                  |
|-----------|-----|-----------|-----------------------|-------------------------|
| BUPNAL    | 233 | 78       | 33.5%                 | [27.4% - 39.5%] |
| CLON      | 178 | 12        | 6.7%                 | [3.1% - 10.4%] |
| **Diff** |     |           | 26.7%                 | [19.6% - 33.8%]   |

**p-value (chi2)** : 1.85e-10
